In [1]:
import pandas as pd;
import numpy as np;
import os;

In [17]:
def generate_report(filepath, CUTOFF):
    #read in file and create DF with numerics
    print(filepath)
    df = pd.read_excel(filepath).fillna(-99)
    df = df.applymap(lambda x: x.replace('%', '') if isinstance(x, str) else x)
    numerics = ["T:COV", "T:MAF", "T:LogR", "ExAC", "gnomADexome", "esp6500si", "1000g2015aug", "ucsf500normN", "ucsf500normT", 'CADD']
    for filter in numerics:
        # try:                
        #     df[filter] = df[filter].str.replace('%', '')
        #     df[filter] = df[filter].astype(float) /100
        # except BaseException:
        #     print(df[filter])
        df[filter] = pd.to_numeric(df[filter], errors='coerce')
    
    
    clean_df = df.copy()
    clean_df.insert(11, "Mut_Cov", (clean_df['T:MAF']/100)*clean_df['T:COV'])   
    clean_df = clean_df[clean_df['ucsf500normN'].isna() | (clean_df['ucsf500normN'] < 0.1)]
    # clean_df = clean_df[
    # (clean_df.gnomADexome < CUTOFF) &
    # (clean_df.esp6500si < CUTOFF) &
    # (clean_df["1000g2015aug"] < CUTOFF) &
    # (clean_df.ucsf500normN < CUTOFF) &
    # (clean_df.ucsf500normT < CUTOFF)]

    #filtering for low percents, based on CUTOFF parameter
    col_names = ['ExAC','gnomADexome','1000g2015aug']
    for col in col_names:
        clean_df = clean_df[clean_df[col]<CUTOFF]



    #ensuring IDH present, which will be reference for filtering in future
    mask = clean_df['Variant'].str.contains("IDH")
    ref_MAF = clean_df.loc[mask,'T:MAF'].iloc[0]
    ref_COV = clean_df.loc[mask,'T:COV'].iloc[0]
    ref_MCov = clean_df.loc[mask,'Mut_Cov'].iloc[0]

    #filtering based on the IDH values
    clean_df = clean_df[clean_df['T:MAF']>= (ref_MAF)]
    clean_df = clean_df[clean_df['T:COV']>= 100]
    clean_df = clean_df[clean_df['Mut_Cov']>= (ref_MCov*.1)]
    clean_df = clean_df[clean_df['TranscriptID'] != -99]

    


    # cols = ["Variant", "TranscriptID", "ExonicFunc", "AAChange"] + numerics[:2] + ["Mut_Cov"] + numerics[2:]
    # report = clean_df.loc[:, cols]
    print("report made")
    return clean_df

In [23]:
#generating for all files in directory
filepaths = os.listdir("Excel")
names = [f.replace('%', '_').split('_')[0] for f in filepaths]
combo = zip(names, filepaths)
reports = {n: generate_report("Excel/" + f, 0.01) for n,f in combo}

#writing csv files to folder
if not os.path.exists("Results"):
    os.mkdir("Results")
for n in reports:
    reports.get(n).to_csv("Results/" + n)
os.listdir("Results")

Excel/CCGL-3149_CGP-9470_T_signout.xlsx
report made
Excel/CCGL-3619_CGP-10495_T_CGP-10511_N_signout.xlsx
report made
Excel/CCGL-3152_CGP-9473_T_signout.xlsx
report made
Excel/CCGL-2835_CGP-8785_T_signout.xlsx
report made
Excel/CCGL-2831_CGP-8784_T_signout.xlsx
report made
Excel/CCGL-3205%2CCCGL-1628_CGP-9586_T_CGP-6259_N_signout.xlsx
report made


['CCGL-3205', 'CCGL-2835', 'CCGL-3619', 'CCGL-3149', 'CCGL-2831', 'CCGL-3152']

In [52]:
variants = set()
for report in reports.values():
    var = report[["Variant", "AAChange"]]
    var = var.reset_index()
    # print(var)
    for index,row in var.iterrows():
        # print(tuple(row[1:]))
        variants.add(tuple(row[1:]))

('SYNE1',
 'SYNE1:NM_033071:exon37:c.4843G>A:p.A1615T,SYNE1:NM_182961:exon37:c.4822G>A:p.A1608T')